### Дипломная работа Александра Соколова

#### Разведывательный анализ данных (EDA)
Кернел 1 из 5 в разделе ML (отредактирован 21.04.2021)
---

# 1. Импорт библиотек, инициализация глобальных констант
## 1.1. Импорт библиотек

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import pandas_profiling
import tqdm

import os
import gc

pd.set_option('display.max_columns', None)

In [ ]:
import utils_21042021 as utils

## 1.2. Глобальные константы

In [ ]:
# CURRENT_DIR = './'  # имя текущей директории для локальной машины 
CURRENT_DIR = '../'  # имя текущей директории для каггл

PATH_TO_WORKDIR = CURRENT_DIR + 'working/'

PATH_TO_TRAIN_TARGET = CURRENT_DIR + 'input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_train_target.csv'
PATH_TO_TEST_TARGET = CURRENT_DIR + 'input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_test_target_contest.csv'

PATH_TO_TRAIN = CURRENT_DIR + 'input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_train_transactions_contest/train_transactions_contest'
PATH_TO_TEST = CURRENT_DIR + 'input/alfabattle2-sandbox/alfabattle2_sand_alfabattle2_test_transactions_contest/test_transactions_contest'


In [ ]:
!pip freeze > requirements.txt

# 2. Вспомогательные функции

In [ ]:
def read_parquet_dataset_from_local(path_to_dataset: str, 
                                    start_from: int = 0,
                                    num_parts_to_read: int = 2, 
                                    columns=None, 
                                    verbose=False,
                                    info_num_parts=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразует их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    list_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset) 
                              if filename.startswith('part')])
    if info_num_parts:
        print(f'Кол-во партиций в папке: {len(list_paths)}')
    start_from = max(0, start_from)
    list_path_to_partitions = list_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for path_to_partition in list_path_to_partitions:
            print(path_to_pirtition)
    for path_to_parquet in tqdm.tqdm_notebook(list_path_to_partitions, 
                                              desc="Читаем файлы:"):
        temp_parquet = pd.read_parquet(path_to_parquet,columns=columns)
        res.append(temp_parquet)
        del temp_parquet
        gc.collect()
    return pd.concat(res).reset_index(drop=True)

# 3. EDA
---
## 3.1 product

In [ ]:
train_targets = pd.read_csv(PATH_TO_TRAIN_TARGET)
train_targets.head()

In [ ]:
pandas_profiling.ProfileReport(train_targets)

В трейне 963811 объектов. 5 видов банковских продуктов. flag бинарный. Пропусков нет.

In [ ]:
utils.simple_plot_barv_count('Распределение кредитных продуктов в трейне по кол-ву', 
                             'product', 
                             train_targets, 
                             1.3, 
                             'Продукт (product)', 
                             'Кол-во')

In [ ]:
utils.simple_plot_barh_procent('Распределение кредитных продуктов в трейне в %-ах', 
                               ['','','','',''], 
                               'product',
                               train_targets)

In [ ]:
temp_df = pd.DataFrame(train_targets['product'].value_counts())
temp_df = temp_df.rename(columns={'product': 'product_count'})
temp_df['product'] = temp_df.index
temp_df['flag'] = train_targets.groupby('product').sum().flag
temp_df['proc_default'] = temp_df['flag']/temp_df['product_count']*100

In [ ]:
utils.simple_plot_barv('Распределение дефолта (%) по кредитным продуктам в трейне',
                       'product',
                       'proc_default', 
                       temp_df, 
                       1.3, 
                       'Продукт (product)', 
                       'Дефолт (%)')

In [ ]:
test_targets = pd.read_csv(PATH_TO_TEST_TARGET)
test_targets.head()

In [ ]:
pandas_profiling.ProfileReport(test_targets)

В тесте 502716 объектов. Также 5 видов банковских продуктов. Пропусков нет.

In [ ]:
utils.simple_plot_barv_count('Распределение кредитных продуктов в тесте по кол-ву', 
                             'product', 
                             test_targets, 
                             1.3, 
                             'Продукт (product)', 
                             'Кол-во')

In [ ]:
utils.simple_plot_barh_procent('Распределение кредитных продуктов в тесте в %-ах', 
                               ['','','','',''], 
                               'product',
                               test_targets)

Распределения признака product в трейне и тесте сбалансированы

## 3.2 Анализ необходимой памяти для выгрузки одного признака

In [ ]:
%%time
temp_df = read_parquet_dataset_from_local(PATH_TO_TRAIN, 
                                                     start_from=0, 
                                                     
                                                     num_parts_to_read=1,
                                                     info_num_parts=True)

memory_usage_of_frame = temp_df.memory_usage(index=True).sum() / 10**9
expected_memory_usage = memory_usage_of_frame * 50
print(f'Объем памяти в  RAM одной партиции данных с транзакциями: {round(memory_usage_of_frame, 3)} Gb')
print(f'Ожидаемый размер в RAM всего датасета: {round(expected_memory_usage, 3)} Gb')

In [ ]:
temp_df.head(10)

In [ ]:
del temp_df
gc.collect()

In [ ]:
%%time
temp_df = read_parquet_dataset_from_local(PATH_TO_TRAIN, 
                                             start_from=0, 
                                             columns = ['app_id'],
                                             num_parts_to_read=50,
                                             info_num_parts=True)

memory_usage_of_frame = temp_df.memory_usage(index=True).sum() / 10**9

print(f'Объем памяти в RAM для одного признака app_id в трейне: {round(memory_usage_of_frame, 3)} Gb')


In [ ]:
%%time
temp_test_df = read_parquet_dataset_from_local(PATH_TO_TEST, 
                                             start_from=0, 
                                             columns = ['app_id'],
                                             num_parts_to_read=50,
                                             info_num_parts=True)

memory_usage_of_frame = temp_test_df.memory_usage(index=True).sum() / 10**9

print(f'Объем памяти в RAM для одного признака app_id в тесте: {round(memory_usage_of_frame, 3)} Gb')

## 3.3 app_id

In [ ]:
temp_df['app_id'].count()

В тренировочной выборке более 270 млн транзакций 

In [ ]:
temp_test_df['app_id'].count()

В тестовой выборке более 170 млн транзакций. В сумме около 444 млн. транзакций. 

In [ ]:
temp_df = pd.DataFrame(temp_df['app_id'].value_counts())
temp_df = temp_df.rename(columns={'app_id': 'app_count'})
temp_df['app_id'] = temp_df.index
temp_df = temp_df.reset_index()

temp_df.app_count.describe()

In [ ]:
temp_test_df = pd.DataFrame(temp_test_df['app_id'].value_counts())
temp_test_df = temp_test_df.rename(columns={'app_id': 'app_count'})
temp_test_df['app_id'] = temp_test_df.index
temp_test_df = temp_test_df.reset_index()

temp_test_df.app_count.describe()

In [ ]:
temp_df.app_count.plot(figsize = (12,6), 
                       title='Распределение кол-ва транзакций по объектам',
                       ylabel = 'Кол-во транзакций')

In [ ]:
del temp_df
del temp_test_df
gc.collect()

## 3.4 amnt

In [ ]:
%%time
temp_df = read_parquet_dataset_from_local(PATH_TO_TRAIN, 
                                             start_from=0, 
                                             columns = ['amnt'],
                                             num_parts_to_read=50,
                                             info_num_parts=True)

memory_usage_of_frame = temp_df.memory_usage(index=True).sum() / 10**9

print(f'Объем памяти в RAM для одного признака amnt в трейне: {round(memory_usage_of_frame, 3)} Gb')


In [ ]:
%%time
temp_test_df = read_parquet_dataset_from_local(PATH_TO_TEST, 
                                             start_from=0, 
                                             columns = ['amnt'],
                                             num_parts_to_read=50,
                                             info_num_parts=True)

memory_usage_of_frame = temp_test_df.memory_usage(index=True).sum() / 10**9

print(f'Объем памяти в RAM для одного признака amnt в тесте: {round(memory_usage_of_frame, 3)} Gb')

In [ ]:
temp_df.amnt.describe()

In [ ]:
temp_test_df.amnt.describe()

In [ ]:
len(temp_df[temp_df['amnt']==0])

In [ ]:
len(temp_test_df[temp_test_df['amnt']==0])

Кол-во пропусков в трейне 1248424 (0.46%), а в тесте 2846900 (1.6%)